# 04_mapping_visualization

## Carbon Sequestration Mapping and Visualization

**Objectives:**
- Generate carbon sequestration maps
- Create interactive dashboards
- Produce publication-quality visualizations
- Export results for GIS software
- Calculate sequestration rates
- Create time-series animations

**Output Formats:** GeoTIFF, Shapefile, HTML, PNG/PDF

## 1. Import Dependencies and Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import json

# Geographic and mapping libraries
import folium
from folium import plugins
import geopandas as gpd
import rasterio
from rasterio.transform import from_origin
import contextily as ctx
from shapely.geometry import Point, Polygon
import pyproj

# Visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.animation as animation
from IPython.display import HTML, display

# Model and data
import joblib
from sklearn.preprocessing import StandardScaler

# Setup
plt.style.use('seaborn-v0_8')
sns.set_palette("viridis")
%matplotlib inline

# Create output directories
import os
os.makedirs('outputs/maps', exist_ok=True)
os.makedirs('outputs/geodata', exist_ok=True)
os.makedirs('outputs/animations', exist_ok=True)
os.makedirs('outputs/dashboards', exist_ok=True)

print("✅ All dependencies imported successfully")
print("✅ Output directories created")

## 2. Load Data and Trained Model

In [ ]:
# Load trained model and preprocessing pipeline
try:
    model = joblib.load('models/carbon_sequestration_model.pkl')
    scaler = joblib.load('models/scaler.pkl')
    feature_engineer = joblib.load('models/feature_engineer.pkl')
    
    with open('models/feature_names.json', 'r') as f:
        feature_names = json.load(f)
    
    print("✅ Successfully loaded trained model and pipeline")
    print(f"📊 Model type: {type(model).__name__}")
    print(f"🔧 Number of features: {len(feature_names)}")
    
except FileNotFoundError:
    print("❌ Model files not found. Please run notebook 03 first.")
    raise

# Load processed data
try:
    biomass_df = pd.read_csv('outputs/biomass_data_with_insights.csv')
    json_df = pd.read_csv('outputs/json_data_with_insights.csv')
    
    # Combine datasets
    spatial_df = pd.concat([biomass_df, json_df], ignore_index=True)
    
    print("✅ Successfully loaded spatial data")
    print(f"📊 Total samples: {len(spatial_df)}")
    
except FileNotFoundError:
    print("❌ Processed data not found. Using sample data...")
    # Create sample spatial data
    np.random.seed(42)
    n_samples = 200
    spatial_df = pd.DataFrame({
        'latitude': np.random.uniform(40.0, 45.0, n_samples),
        'longitude': np.random.uniform(-75.0, -70.0, n_samples),
        'biomass': np.random.normal(2.5, 1.0, n_samples).clip(0.1, 5.0),
        'carbon_stock': np.random.normal(1.2, 0.5, n_samples).clip(0.05, 2.5)
    })

# Display data summary
print("\n📊 Spatial Data Summary:")
print(f"Geographic bounds:")
print(f"  Latitude: {spatial_df['latitude'].min():.4f} to {spatial_df['latitude'].max():.4f}")
print(f"  Longitude: {spatial_df['longitude'].min():.4f} to {spatial_df['longitude'].max():.4f}")
print(f"  Carbon stock: {spatial_df['carbon_stock'].min():.3f} to {spatial_df['carbon_stock'].max():.3f} kg/m²")

# Convert to GeoDataFrame
geometry = [Point(xy) for xy in zip(spatial_df['longitude'], spatial_df['latitude'])]
gdf = gpd.GeoDataFrame(spatial_df, geometry=geometry, crs="EPSG:4326")

print(f"\n🌍 GeoDataFrame created with CRS: {gdf.crs}")

## 3. Carbon Stock Mapping

In [ ]:
class CarbonStockMapper:
    """Create comprehensive carbon stock maps."""
    
    def __init__(self, gdf):
        self.gdf = gdf
        self.figures = {}
    
    def create_static_maps(self):
        """Create static carbon stock maps."""
        
        print("🗺️ Creating static carbon stock maps...")
        
        # Create subplots
        fig, axes = plt.subplots(2, 2, figsize=(20, 16))
        fig.suptitle('Carbon Stock Spatial Distribution', fontsize=20, fontweight='bold')
        
        # 1. Scatter plot
        scatter = axes[0, 0].scatter(
            self.gdf['longitude'], self.gdf['latitude'], 
            c=self.gdf['carbon_stock'], 
            cmap='YlGn', s=50, alpha=0.7
        )
        axes[0, 0].set_title('Carbon Stock Distribution', fontsize=14, fontweight='bold')
        axes[0, 0].set_xlabel('Longitude')
        axes[0, 0].set_ylabel('Latitude')
        plt.colorbar(scatter, ax=axes[0, 0], label='Carbon Stock (kg/m²)')
        
        # 2. Hexbin plot
        hexbin = axes[0, 1].hexbin(
            self.gdf['longitude'], self.gdf['latitude'], 
            C=self.gdf['carbon_stock'], 
            gridsize=20, cmap='YlGn', reduce_C_function=np.mean
        )
        axes[0, 1].set_title('Carbon Stock Density', fontsize=14, fontweight='bold')
        axes[0, 1].set_xlabel('Longitude')
        axes[0, 1].set_ylabel('Latitude')
        plt.colorbar(hexbin, ax=axes[0, 1], label='Carbon Stock (kg/m²)')
        
        # 3. Contour plot
        from scipy.interpolate import griddata
        
        # Create grid for contour
        x = self.gdf['longitude']
        y = self.gdf['latitude']
        z = self.gdf['carbon_stock']
        
        xi = np.linspace(x.min(), x.max(), 100)
        yi = np.linspace(y.min(), y.max(), 100)
        xi, yi = np.meshgrid(xi, yi)
        
        # Interpolate
        zi = griddata((x, y), z, (xi, yi), method='cubic')
        
        contour = axes[1, 0].contourf(xi, yi, zi, levels=15, cmap='YlGn', alpha=0.8)
        axes[1, 0].scatter(x, y, c=z, s=30, edgecolor='black', alpha=0.6)
        axes[1, 0].set_title('Carbon Stock Contour Map', fontsize=14, fontweight='bold')
        axes[1, 0].set_xlabel('Longitude')
        axes[1, 0].set_ylabel('Latitude')
        plt.colorbar(contour, ax=axes[1, 0], label='Carbon Stock (kg/m²)')
        
        # 4. Classification map
        carbon_classes = pd.cut(
            self.gdf['carbon_stock'], 
            bins=[0, 0.5, 1.0, 1.5, 2.0, np.inf],
            labels=['Very Low', 'Low', 'Medium', 'High', 'Very High']
        )
        
        colors = ['#ffffcc', '#c2e699', '#78c679', '#31a354', '#006837']
        
        for i, cls in enumerate(['Very Low', 'Low', 'Medium', 'High', 'Very High']):
            mask = carbon_classes == cls
            axes[1, 1].scatter(
                self.gdf[mask]['longitude'], self.gdf[mask]['latitude'],
                c=colors[i], label=cls, s=50, alpha=0.7
            )
        
        axes[1, 1].set_title('Carbon Stock Classification', fontsize=14, fontweight='bold')
        axes[1, 1].set_xlabel('Longitude')
        axes[1, 1].set_ylabel('Latitude')
        axes[1, 1].legend()
        
        plt.tight_layout()
        plt.savefig('outputs/maps/carbon_stock_static_maps.png', dpi=300, bbox_inches='tight')
        plt.savefig('outputs/maps/carbon_stock_static_maps.pdf', bbox_inches='tight')
        plt.show()
        
        self.figures['static'] = fig
        print("✅ Static maps saved to outputs/maps/")
    
    def create_interactive_plotly_maps(self):
        """Create interactive Plotly maps."""
        
        print("\n🌐 Creating interactive Plotly maps...")
        
        # Create subplots
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                'Carbon Stock Distribution',
                'Carbon Stock Density',
                'Carbon Stock Interpolation',
                'Carbon Stock Classification'
            ),
            specs=[[{"type": "scatter"}, {"type": "densitymapbox"}],
                   [{"type": "contour"}, {"type": "scatter"}]]
        )
        
        # 1. Scatter plot
        fig.add_trace(
            go.Scattermapbox(
                lat=self.gdf['latitude'],
                lon=self.gdf['longitude'],
                mode='markers',
                marker=dict(
                    size=8,
                    color=self.gdf['carbon_stock'],
                    colorscale='YlGn',
                    showscale=True,
                    colorbar=dict(title='Carbon Stock')
                ),
                text=[f"Carbon: {c:.2f} kg/m²" for c in self.gdf['carbon_stock']],
                hovertemplate='<b>Lat</b>: %{lat}<br><b>Lon</b>: %{lon}<br>%{text}<extra></extra>',
                name='Measurement Points'
            ),
            row=1, col=1
        )
        
        # 2. Density map
        fig.add_trace(
            go.Densitymapbox(
                lat=self.gdf['latitude'],
                lon=self.gdf['longitude'],
                z=self.gdf['carbon_stock'],
                radius=20,
                colorscale='YlGn',
                colorbar=dict(title='Carbon Density'),
                name='Carbon Density'
            ),
            row=1, col=2
        )
        
        # 3. Contour plot (simplified for Plotly)
        fig.add_trace(
            go.Contour(
                x=self.gdf['longitude'],
                y=self.gdf['latitude'],
                z=self.gdf['carbon_stock'],
                colorscale='YlGn',
                connectgaps=True,
                colorbar=dict(title='Carbon Stock'),
                name='Interpolated'
            ),
            row=2, col=1
        )
        
        # 4. Classification scatter
        carbon_classes = pd.cut(
            self.gdf['carbon_stock'], 
            bins=[0, 0.5, 1.0, 1.5, 2.0, np.inf],
            labels=['Very Low', 'Low', 'Medium', 'High', 'Very High']
        )
        
        colors = ['#ffffcc', '#c2e699', '#78c679', '#31a354', '#006837']
        
        for i, cls in enumerate(['Very Low', 'Low', 'Medium', 'High', 'Very High']):
            mask = carbon_classes == cls
            fig.add_trace(
                go.Scatter(
                    x=self.gdf[mask]['longitude'],
                    y=self.gdf[mask]['latitude'],
                    mode='markers',
                    marker=dict(color=colors[i], size=8),
                    name=cls,
                    showlegend=True
                ),
                row=2, col=2
            )
        
        # Update layout
        fig.update_layout(
            title_text="Interactive Carbon Stock Maps",
            height=800,
            showlegend=True
        )
        
        # Update mapbox settings
        fig.update_mapboxes(
            style="open-street-map",
            center=dict(
                lat=self.gdf['latitude'].mean(),
                lon=self.gdf['longitude'].mean()
            ),
            zoom=9
        )
        
        fig.show()
        
        # Save as HTML
        fig.write_html('outputs/maps/interactive_carbon_maps.html')
        self.figures['interactive'] = fig
        print("✅ Interactive maps saved to outputs/maps/interactive_carbon_maps.html")

# Create carbon stock maps
mapper = CarbonStockMapper(gdf)
mapper.create_static_maps()
mapper.create_interactive_plotly_maps()

## 4. Sequestration Rate Calculations

In [ ]:
class SequestrationCalculator:
    """Calculate carbon sequestration rates and potentials."""
    
    def __init__(self, gdf):
        self.gdf = gdf
        self.sequestration_rates = None
    
    def calculate_sequestration_rates(self, growth_rate=0.05, conversion_factor=0.47):
        """Calculate carbon sequestration rates based on biomass growth."""
        
        print("📈 Calculating carbon sequestration rates...")
        
        # Assume annual biomass growth rate (5% default)
        # Convert biomass growth to carbon sequestration
        
        sequestration_data = []
        
        for idx, row in self.gdf.iterrows():
            current_biomass = row['biomass'] if 'biomass' in row else row.get('biomass_value', 2.0)
            current_carbon = row['carbon_stock']
            
            # Calculate annual sequestration
            annual_biomass_growth = current_biomass * growth_rate
            annual_carbon_sequestration = annual_biomass_growth * conversion_factor
            
            # Calculate potential carbon stock in 10 years
            future_biomass = current_biomass * (1 + growth_rate) ** 10
            future_carbon = future_biomass * conversion_factor
            
            sequestration_data.append({
                'geometry': row['geometry'],
                'latitude': row['latitude'],
                'longitude': row['longitude'],
                'current_biomass': current_biomass,
                'current_carbon': current_carbon,
                'annual_sequestration': annual_carbon_sequestration,
                'future_carbon_10yr': future_carbon,
                'sequestration_potential': future_carbon - current_carbon
            })
        
        self.sequestration_rates = gpd.GeoDataFrame(sequestration_data, crs="EPSG:4326")
        
        print("✅ Sequestration rates calculated")
        print(f"📊 Annual sequestration range: {self.sequestration_rates['annual_sequestration'].min():.4f} to {self.sequestration_rates['annual_sequestration'].max():.4f} kg/m²/yr")
        
        return self.sequestration_rates
    
    def create_sequestration_maps(self):
        """Create maps showing sequestration rates and potentials."""
        
        if self.sequestration_rates is None:
            print("❌ Please calculate sequestration rates first")
            return
        
        print("\n🗺️ Creating sequestration rate maps...")
        
        # Create subplots
        fig, axes = plt.subplots(2, 2, figsize=(20, 16))
        fig.suptitle('Carbon Sequestration Analysis', fontsize=20, fontweight='bold')
        
        # 1. Annual sequestration rates
        sc1 = axes[0, 0].scatter(
            self.sequestration_rates['longitude'], 
            self.sequestration_rates['latitude'],
            c=self.sequestration_rates['annual_sequestration'],
            cmap='RdYlGn', s=50, alpha=0.7
        )
        axes[0, 0].set_title('Annual Sequestration Rates', fontsize=14, fontweight='bold')
        axes[0, 0].set_xlabel('Longitude')
        axes[0, 0].set_ylabel('Latitude')
        plt.colorbar(sc1, ax=axes[0, 0], label='Sequestration Rate (kg/m²/yr)')
        
        # 2. Sequestration potential
        sc2 = axes[0, 1].scatter(
            self.sequestration_rates['longitude'], 
            self.sequestration_rates['latitude'],
            c=self.sequestration_rates['sequestration_potential'],
            cmap='viridis', s=50, alpha=0.7
        )
        axes[0, 1].set_title('10-Year Sequestration Potential', fontsize=14, fontweight='bold')
        axes[0, 1].set_xlabel('Longitude')
        axes[0, 1].set_ylabel('Latitude')
        plt.colorbar(sc2, ax=axes[0, 1], label='Carbon Potential (kg/m²)')
        
        # 3. Current vs Future carbon
        axes[1, 0].scatter(
            self.sequestration_rates['current_carbon'],
            self.sequestration_rates['future_carbon_10yr'],
            c=self.sequestration_rates['annual_sequestration'],
            cmap='RdYlGn', s=50, alpha=0.7
        )
        axes[1, 0].plot([0, 3], [0, 3], 'r--', alpha=0.7, label='1:1 Line')
        axes[1, 0].set_title('Current vs Future Carbon Stock', fontsize=14, fontweight='bold')
        axes[1, 0].set_xlabel('Current Carbon (kg/m²)')
        axes[1, 0].set_ylabel('Future Carbon (kg/m²)')
        axes[1, 0].legend()
        
        # 4. Sequestration rate distribution
        sequestration_rates = self.sequestration_rates['annual_sequestration']
        axes[1, 1].hist(sequestration_rates, bins=20, color='lightblue', edgecolor='black', alpha=0.7)
        axes[1, 1].axvline(sequestration_rates.mean(), color='red', linestyle='--', 
                          label=f'Mean: {sequestration_rates.mean():.4f}')
        axes[1, 1].set_title('Sequestration Rate Distribution', fontsize=14, fontweight='bold')
        axes[1, 1].set_xlabel('Sequestration Rate (kg/m²/yr)')
        axes[1, 1].set_ylabel('Frequency')
        axes[1, 1].legend()
        
        plt.tight_layout()
        plt.savefig('outputs/maps/sequestration_analysis.png', dpi=300, bbox_inches='tight')
        plt.savefig('outputs/maps/sequestration_analysis.pdf', bbox_inches='tight')
        plt.show()
        
        # Create interactive sequestration map
        fig_int = px.scatter_mapbox(
            self.sequestration_rates,
            lat='latitude',
            lon='longitude',
            color='annual_sequestration',
            size='sequestration_potential',
            hover_data={
                'current_carbon': ':.3f',
                'annual_sequestration': ':.4f',
                'sequestration_potential': ':.3f'
            },
            color_continuous_scale='RdYlGn',
            title='Carbon Sequestration Rates and Potentials',
            zoom=9
        )
        
        fig_int.update_layout(mapbox_style="open-street-map")
        fig_int.update_layout(margin={"r":0,"t":30,"l":0,"b":0})
        
        fig_int.show()
        fig_int.write_html('outputs/maps/interactive_sequestration_map.html')
        
        print("✅ Sequestration maps saved to outputs/maps/")
    
    def generate_sequestration_report(self):
        """Generate comprehensive sequestration report."""
        
        if self.sequestration_rates is None:
            print("❌ Please calculate sequestration rates first")
            return
        
        print("\n" + "="*80)
        print("📊 CARBON SEQUESTRATION ANALYSIS REPORT")
        print("="*80)
        
        total_area_km2 = len(self.sequestration_rates) * 0.01  # Assuming 100m² per sample
        
        # Calculate totals
        total_current_carbon = self.sequestration_rates['current_carbon'].sum() * 10000  # kg/ha
        total_annual_sequestration = self.sequestration_rates['annual_sequestration'].sum() * 10000  # kg/ha/yr
        total_10yr_potential = self.sequestration_rates['sequestration_potential'].sum() * 10000  # kg/ha
        
        print(f"\n🌳 Study Area Summary:")
        print(f"   Sample points: {len(self.sequestration_rates)}")
        print(f"   Estimated area: {total_area_km2:.2f} km²")
        
        print(f"\n📈 Carbon Stocks:")
        print(f"   Current carbon stock: {total_current_carbon/1000:.1f} tonnes")
        print(f"   Average carbon density: {self.sequestration_rates['current_carbon'].mean():.3f} kg/m²")
        
        print(f"\n🎯 Sequestration Potential:")
        print(f"   Annual sequestration: {total_annual_sequestration/1000:.1f} tonnes/year")
        print(f"   10-year potential: {total_10yr_potential/1000:.1f} tonnes")
        print(f"   Average sequestration rate: {self.sequestration_rates['annual_sequestration'].mean():.4f} kg/m²/yr")
        
        print(f"\n📊 Distribution Analysis:")
        print(f"   Min sequestration rate: {self.sequestration_rates['annual_sequestration'].min():.4f} kg/m²/yr")
        print(f"   Max sequestration rate: {self.sequestration_rates['annual_sequestration'].max():.4f} kg/m²/yr")
        print(f"   Coefficient of variation: {self.sequestration_rates['annual_sequestration'].std() / self.sequestration_rates['annual_sequestration'].mean() * 100:.1f}%")
        
        # Identify hotspots
        hotspot_threshold = self.sequestration_rates['annual_sequestration'].quantile(0.75)
        hotspots = self.sequestration_rates[self.sequestration_rates['annual_sequestration'] > hotspot_threshold]
        
        print(f"\n🔥 Sequestration Hotspots:")
        print(f"   Hotspot threshold: >{hotspot_threshold:.4f} kg/m²/yr")
        print(f"   Number of hotspots: {len(hotspots)} ({len(hotspots)/len(self.sequestration_rates)*100:.1f}% of area)")
        print(f"   Hotspot average rate: {hotspots['annual_sequestration'].mean():.4f} kg/m²/yr")
        
        print("\n" + "="*80)

# Calculate and visualize sequestration rates
sequestration_calc = SequestrationCalculator(gdf)
sequestration_rates = sequestration_calc.calculate_sequestration_rates()
sequestration_calc.create_sequestration_maps()
sequestration_calc.generate_sequestration_report()

## 5. Interactive Web Maps with Folium

In [ ]:
class InteractiveMapCreator:
    """Create interactive web maps with multiple layers and controls."""
    
    def __init__(self, gdf, sequestration_rates=None):
        self.gdf = gdf
        self.sequestration_rates = sequestration_rates
        self.maps = {}
    
    def create_carbon_stock_map(self):
        """Create interactive carbon stock map."""
        
        print("🌍 Creating interactive carbon stock map...")
        
        # Calculate center of the data
        center_lat = self.gdf['latitude'].mean()
        center_lon = self.gdf['longitude'].mean()
        
        # Create base map
        carbon_map = folium.Map(
            location=[center_lat, center_lon], 
            zoom_start=10,
            tiles='OpenStreetMap'
        )
        
        # Add carbon stock points
        for idx, row in self.gdf.iterrows():
            carbon_val = row['carbon_stock']
            
            # Determine color based on carbon value
            if carbon_val < 0.5:
                color = 'lightgray'
            elif carbon_val < 1.0:
                color = 'lightgreen'
            elif carbon_val < 1.5:
                color = 'green'
            elif carbon_val < 2.0:
                color = 'darkgreen'
            else:
                color = 'red'
            
            # Create popup text
            popup_text = f"""
            <b>Carbon Stock Information</b><br>
            Carbon: {carbon_val:.3f} kg/m²<br>
            Latitude: {row['latitude']:.4f}<br>
            Longitude: {row['longitude']:.4f}<br>
            """
            
            if 'biomass' in row:
                popup_text += f"Biomass: {row['biomass']:.3f} kg/m²<br>"
            
            # Add marker
            folium.CircleMarker(
                location=[row['latitude'], row['longitude']],
                radius=8,
                popup=folium.Popup(popup_text, max_width=300),
                tooltip=f"Carbon: {carbon_val:.3f} kg/m²",
                color=color,
                fillColor=color,
                fillOpacity=0.7,
                weight=1
            ).add_to(carbon_map)
        
        # Add heatmap layer
        heat_data = [[row['latitude'], row['longitude'], row['carbon_stock']] for idx, row in self.gdf.iterrows()]
        plugins.HeatMap(heat_data, name='Carbon Stock Heatmap', min_opacity=0.3, max_zoom=18).add_to(carbon_map)
        
        # Add layer control
        folium.LayerControl().add_to(carbon_map)
        
        # Add title
        title_html = '''
                 <h3 align="center" style="font-size:20px"><b>Carbon Stock Distribution Map</b></h3>
                 '''
        carbon_map.get_root().html.add_child(folium.Element(title_html))
        
        self.maps['carbon_stock'] = carbon_map
        carbon_map.save('outputs/maps/interactive_carbon_stock_map.html')
        print("✅ Carbon stock map saved to outputs/maps/interactive_carbon_stock_map.html")
        
        return carbon_map
    
    def create_sequestration_map(self):
        """Create interactive sequestration map."""
        
        if self.sequestration_rates is None:
            print("❌ No sequestration data available")
            return None
        
        print("🌍 Creating interactive sequestration map...")
        
        center_lat = self.sequestration_rates['latitude'].mean()
        center_lon = self.sequestration_rates['longitude'].mean()
        
        sequestration_map = folium.Map(
            location=[center_lat, center_lon], 
            zoom_start=10,
            tiles='OpenStreetMap'
        )
        
        # Add sequestration points
        for idx, row in self.sequestration_rates.iterrows():
            seq_rate = row['annual_sequestration']
            seq_potential = row['sequestration_potential']
            
            # Determine color based on sequestration rate
            if seq_rate < 0.02:
                color = 'lightgray'
            elif seq_rate < 0.05:
                color = 'lightgreen'
            elif seq_rate < 0.08:
                color = 'green'
            elif seq_rate < 0.12:
                color = 'orange'
            else:
                color = 'red'
            
            # Size based on potential
            radius = 5 + (seq_potential * 10)
            
            popup_text = f"""
            <b>Sequestration Information</b><br>
            Annual Rate: {seq_rate:.4f} kg/m²/yr<br>
            10-Year Potential: {seq_potential:.3f} kg/m²<br>
            Current Carbon: {row['current_carbon']:.3f} kg/m²<br>
            Future Carbon: {row['future_carbon_10yr']:.3f} kg/m²<br>
            """
            
            folium.CircleMarker(
                location=[row['latitude'], row['longitude']],
                radius=radius,
                popup=folium.Popup(popup_text, max_width=300),
                tooltip=f"Seq Rate: {seq_rate:.4f} kg/m²/yr",
                color=color,
                fillColor=color,
                fillOpacity=0.7,
                weight=1
            ).add_to(sequestration_map)
        
        # Add sequestration heatmap
        seq_heat_data = [[row['latitude'], row['longitude'], row['annual_sequestration']] 
                        for idx, row in self.sequestration_rates.iterrows()]
        plugins.HeatMap(seq_heat_data, name='Sequestration Rate Heatmap', 
                       min_opacity=0.3, max_zoom=18, gradient={0.4: 'blue', 0.65: 'lime', 1: 'red'}).add_to(sequestration_map)
        
        # Add layer control
        folium.LayerControl().add_to(sequestration_map)
        
        title_html = '''
                 <h3 align="center" style="font-size:20px"><b>Carbon Sequestration Rates Map</b></h3>
                 '''
        sequestration_map.get_root().html.add_child(folium.Element(title_html))
        
        self.maps['sequestration'] = sequestration_map
        sequestration_map.save('outputs/maps/interactive_sequestration_map.html')
        print("✅ Sequestration map saved to outputs/maps/interactive_sequestration_map.html")
        
        return sequestration_map
    
    def create_comparison_dashboard(self):
        """Create a dashboard with multiple maps."""
        
        print("\n📊 Creating comparison dashboard...")
        
        center_lat = self.gdf['latitude'].mean()
        center_lon = self.gdf['longitude'].mean()
        
        # Create base map
        dashboard = folium.Map(
            location=[center_lat, center_lon], 
            zoom_start=10,
            tiles='OpenStreetMap'
        )
        
        # Add carbon stock layer
        carbon_layer = folium.FeatureGroup(name='Carbon Stock')
        for idx, row in self.gdf.iterrows():
            folium.CircleMarker(
                location=[row['latitude'], row['longitude']],
                radius=6,
                popup=f"Carbon: {row['carbon_stock']:.3f} kg/m²",
                color='green',
                fillColor='green',
                fillOpacity=0.6
            ).add_to(carbon_layer)
        carbon_layer.add_to(dashboard)
        
        # Add sequestration layer if available
        if self.sequestration_rates is not None:
            seq_layer = folium.FeatureGroup(name='Sequestration Rates')
            for idx, row in self.sequestration_rates.iterrows():
                folium.CircleMarker(
                    location=[row['latitude'], row['longitude']],
                    radius=6,
                    popup=f"Seq Rate: {row['annual_sequestration']:.4f} kg/m²/yr",
                    color='blue',
                    fillColor='blue',
                    fillOpacity=0.6
                ).add_to(seq_layer)
            seq_layer.add_to(dashboard)
        
        # Add heatmaps
        carbon_heat_data = [[row['latitude'], row['longitude'], row['carbon_stock']] for idx, row in self.gdf.iterrows()]
        plugins.HeatMap(carbon_heat_data, name='Carbon Heatmap', min_opacity=0.3).add_to(dashboard)
        
        if self.sequestration_rates is not None:
            seq_heat_data = [[row['latitude'], row['longitude'], row['annual_sequestration']] 
                            for idx, row in self.sequestration_rates.iterrows()]
            plugins.HeatMap(seq_heat_data, name='Sequestration Heatmap', min_opacity=0.3).add_to(dashboard)
        
        # Add layer control
        folium.LayerControl().add_to(dashboard)
        
        # Add minimap
        plugins.MiniMap().add_to(dashboard)
        
        # Add fullscreen button
        plugins.Fullscreen().add_to(dashboard)
        
        title_html = '''
                 <h3 align="center" style="font-size:20px"><b>Carbon Sequestration Dashboard</b></h3>
                 '''
        dashboard.get_root().html.add_child(folium.Element(title_html))
        
        self.maps['dashboard'] = dashboard
        dashboard.save('outputs/dashboards/carbon_sequestration_dashboard.html')
        print("✅ Dashboard saved to outputs/dashboards/carbon_sequestration_dashboard.html")
        
        return dashboard

# Create interactive maps
map_creator = InteractiveMapCreator(gdf, sequestration_rates)
carbon_map = map_creator.create_carbon_stock_map()
seq_map = map_creator.create_sequestration_map()
dashboard = map_creator.create_comparison_dashboard()

# Display one map in notebook
print("\n📍 Displaying carbon stock map in notebook...")
display(carbon_map)

## 6. Time-series Animation

In [ ]:
class TimeSeriesAnimator:
    """Create time-series animations of carbon sequestration."""
    
    def __init__(self, sequestration_rates):
        self.sequestration_rates = sequestration_rates
    
    def create_sequestration_animation(self, years=10):
        """Create animation showing carbon accumulation over time."""
        
        print("🎬 Creating sequestration time-series animation...")
        
        # Create figure
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        
        # Initialize scatter plots
        sc1 = ax1.scatter([], [], c=[], cmap='YlGn', s=50, alpha=0.7)
        sc2 = ax2.scatter([], [], c=[], cmap='viridis', s=50, alpha=0.7)
        
        # Set up axes
        ax1.set_xlim(self.sequestration_rates['longitude'].min(), self.sequestration_rates['longitude'].max())
        ax1.set_ylim(self.sequestration_rates['latitude'].min(), self.sequestration_rates['latitude'].max())
        ax1.set_xlabel('Longitude')
        ax1.set_ylabel('Latitude')
        ax1.set_title('Carbon Stock Over Time')
        
        ax2.set_xlim(self.sequestration_rates['longitude'].min(), self.sequestration_rates['longitude'].max())
        ax2.set_ylim(self.sequestration_rates['latitude'].min(), self.sequestration_rates['latitude'].max())
        ax2.set_xlabel('Longitude')
        ax2.set_ylabel('Latitude')
        ax2.set_title('Cumulative Sequestration')
        
        # Add colorbars
        plt.colorbar(sc1, ax=ax1, label='Carbon Stock (kg/m²)')
        plt.colorbar(sc2, ax=ax2, label='Cumulative Seq. (kg/m²)')
        
        def animate(frame):
            year = frame + 1
            
            # Calculate carbon stock for this year
            current_carbon = self.sequestration_rates['current_carbon'].values
            annual_seq = self.sequestration_rates['annual_sequestration'].values
            carbon_at_year = current_carbon + (annual_seq * year)
            cumulative_seq = annual_seq * year
            
            # Update scatter plots
            sc1.set_offsets(np.column_stack([self.sequestration_rates['longitude'], 
                                           self.sequestration_rates['latitude']]))
            sc1.set_array(carbon_at_year)
            sc1.set_clim(0, carbon_at_year.max())
            
            sc2.set_offsets(np.column_stack([self.sequestration_rates['longitude'], 
                                           self.sequestration_rates['latitude']]))
            sc2.set_array(cumulative_seq)
            sc2.set_clim(0, cumulative_seq.max())
            
            ax1.set_title(f'Carbon Stock - Year {year}')
            ax2.set_title(f'Cumulative Sequestration - Year {year}')
            
            return sc1, sc2
        
        # Create animation
        anim = animation.FuncAnimation(
            fig, animate, frames=years, interval=1000, blit=False, repeat=True
        )
        
        # Save animation
        anim.save('outputs/animations/sequestration_animation.gif', writer='pillow', fps=1)
        
        plt.tight_layout()
        plt.show()
        
        print("✅ Animation saved to outputs/animations/sequestration_animation.gif")
        
        return anim
    
    def create_aggregate_timeseries(self, years=20):
        """Create aggregate time-series plot of carbon accumulation."""
        
        print("\n📈 Creating aggregate time-series plot...")
        
        years_range = range(years + 1)
        
        # Calculate aggregate metrics over time
        total_carbon = []
        total_sequestration = []
        mean_carbon = []
        
        for year in years_range:
            carbon_at_year = (self.sequestration_rates['current_carbon'] + 
                            self.sequestration_rates['annual_sequestration'] * year)
            sequestration_at_year = self.sequestration_rates['annual_sequestration'] * year
            
            total_carbon.append(carbon_at_year.sum() * 10000)  # Convert to kg/ha
            total_sequestration.append(sequestration_at_year.sum() * 10000)  # Convert to kg/ha
            mean_carbon.append(carbon_at_year.mean())
        
        # Create plot
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        
        # Total carbon and sequestration
        ax1.plot(years_range, np.array(total_carbon) / 1000, 'g-', linewidth=2, label='Total Carbon')
        ax1.plot(years_range, np.array(total_sequestration) / 1000, 'b--', linewidth=2, label='Cumulative Sequestration')
        ax1.set_xlabel('Years')
        ax1.set_ylabel('Carbon (tonnes)')
        ax1.set_title('Total Carbon Stock and Sequestration')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Mean carbon density
        ax2.plot(years_range, mean_carbon, 'r-', linewidth=2)
        ax2.set_xlabel('Years')
        ax2.set_ylabel('Mean Carbon Density (kg/m²)')
        ax2.set_title('Mean Carbon Density Over Time')
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('outputs/animations/aggregate_timeseries.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        print("✅ Aggregate time-series plot saved to outputs/animations/aggregate_timeseries.png")

# Create animations and time-series
if sequestration_rates is not None:
    animator = TimeSeriesAnimator(sequestration_rates)
    anim = animator.create_sequestration_animation(years=10)
    animator.create_aggregate_timeseries(years=20)
else:
    print("❌ No sequestration data available for animation")

## 7. Export to GIS Formats

In [ ]:
class GISExporter:
    """Export data to various GIS formats."""
    
    def __init__(self, gdf, sequestration_rates=None):
        self.gdf = gdf
        self.sequestration_rates = sequestration_rates
    
    def export_to_shapefile(self):
        """Export data to ESRI Shapefile format."""
        
        print("🗂️ Exporting to Shapefile...")
        
        # Export carbon stock data
        carbon_gdf = self.gdf.copy()
        carbon_gdf.to_file('outputs/geodata/carbon_stock_data.shp', driver='ESRI Shapefile')
        
        # Export sequestration data if available
        if self.sequestration_rates is not None:
            seq_gdf = self.sequestration_rates.copy()
            seq_gdf.to_file('outputs/geodata/sequestration_data.shp', driver='ESRI Shapefile')
        
        print("✅ Shapefiles exported to outputs/geodata/")
    
    def export_to_geotiff(self):
        """Export raster data to GeoTIFF format."""
        
        print("\n🗂️ Exporting to GeoTIFF...")
        
        def create_raster_from_points(gdf, value_column, output_path, resolution=0.001):
            """Create raster from point data using interpolation."""
            
            from scipy.interpolate import griddata
            
            # Create grid
            x = gdf['longitude'].values
            y = gdf['latitude'].values
            z = gdf[value_column].values
            
            xi = np.linspace(x.min(), x.max(), int((x.max() - x.min()) / resolution))
            yi = np.linspace(y.min(), y.max(), int((y.max() - y.min()) / resolution))
            xi, yi = np.meshgrid(xi, yi)
            
            # Interpolate
            zi = griddata((x, y), z, (xi, yi), method='linear')
            
            # Set up transform
            transform = from_origin(xi.min(), yi.max(), resolution, resolution)
            
            # Write to GeoTIFF
            with rasterio.open(
                output_path,
                'w',
                driver='GTiff',
                height=zi.shape[0],
                width=zi.shape[1],
                count=1,
                dtype=zi.dtype,
                crs='EPSG:4326',
                transform=transform,
            ) as dst:
                dst.write(zi, 1)
        
        # Export carbon stock raster
        create_raster_from_points(self.gdf, 'carbon_stock', 
                                'outputs/geodata/carbon_stock.tif')
        
        # Export sequestration raster if available
        if self.sequestration_rates is not None:
            create_raster_from_points(self.sequestration_rates, 'annual_sequestration',
                                    'outputs/geodata/sequestration_rates.tif')
        
        print("✅ GeoTIFF files exported to outputs/geodata/")
    
    def export_to_geojson(self):
        """Export data to GeoJSON format."""
        
        print("\n🗂️ Exporting to GeoJSON...")
        
        # Export carbon stock data
        self.gdf.to_file('outputs/geodata/carbon_stock_data.geojson', driver='GeoJSON')
        
        # Export sequestration data if available
        if self.sequestration_rates is not None:
            self.sequestration_rates.to_file('outputs/geodata/sequestration_data.geojson', driver='GeoJSON')
        
        print("✅ GeoJSON files exported to outputs/geodata/")
    
    def export_to_kml(self):
        """Export data to KML format for Google Earth."""
        
        print("\n🗂️ Exporting to KML...")
        
        try:
            # Export carbon stock data
            carbon_gdf = self.gdf.copy()
            
            # Add description for KML
            carbon_gdf['description'] = carbon_gdf.apply(
                lambda row: f"Carbon: {row['carbon_stock']:.3f} kg/m²", axis=1
            )
            
            carbon_gdf.to_file('outputs/geodata/carbon_stock_data.kml', driver='KML')
            
            # Export sequestration data if available
            if self.sequestration_rates is not None:
                seq_gdf = self.sequestration_rates.copy()
                seq_gdf['description'] = seq_gdf.apply(
                    lambda row: f"Seq Rate: {row['annual_sequestration']:.4f} kg/m²/yr", axis=1
                )
                seq_gdf.to_file('outputs/geodata/sequestration_data.kml', driver='KML')
            
            print("✅ KML files exported to outputs/geodata/")
            
        except Exception as e:
            print(f"⚠️ KML export failed: {e}")
            print("💡 Install 'fiona' with KML support: pip install fiona")
    
    def create_export_summary(self):
        """Create summary of exported files."""
        
        print("\n" + "="*80)
        print("📁 GIS EXPORT SUMMARY")
        print("="*80)
        
        export_dir = 'outputs/geodata/'
        files = os.listdir(export_dir)
        
        print(f"\nExported files in {export_dir}:")
        for file in sorted(files):
            file_path = os.path.join(export_dir, file)
            file_size = os.path.getsize(file_path) / 1024  # KB
            print(f"  {file} ({file_size:.1f} KB)")
        
        print(f"\n📊 Total files exported: {len(files)}")
        
        # Format-specific information
        print(f"\n🔧 Supported GIS Software:")
        print(f"  • Shapefile: ArcGIS, QGIS, ArcMap")
        print(f"  • GeoTIFF: Most raster GIS software")
        print(f"  • GeoJSON: Web mapping, QGIS, ArcGIS Pro")
        print(f"  • KML: Google Earth, Google Maps")

# Export to all formats
gis_exporter = GISExporter(gdf, sequestration_rates)
gis_exporter.export_to_shapefile()
gis_exporter.export_to_geotiff()
gis_exporter.export_to_geojson()
gis_exporter.export_to_kml()
gis_exporter.create_export_summary()

## 8. Final Summary and Visualization Gallery

In [ ]:
def create_visualization_gallery():
    """Create a summary of all visualizations and exports."""
    
    print("="*80)
    print("🎨 VISUALIZATION AND MAPPING GALLERY")
    print("="*80)
    
    # Count files in each output directory
    output_dirs = {
        'Static Maps': 'outputs/maps/',
        'Interactive Maps': 'outputs/maps/',
        'Dashboards': 'outputs/dashboards/',
        'Animations': 'outputs/animations/',
        'GIS Data': 'outputs/geodata/'
    }
    
    total_files = 0
    
    print("\n📁 OUTPUT FILES SUMMARY:")
    for category, directory in output_dirs.items():
        if os.path.exists(directory):
            files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
            file_count = len(files)
            total_files += file_count
            
            # Get file types
            extensions = {}
            for file in files:
                ext = os.path.splitext(file)[1].lower()
                extensions[ext] = extensions.get(ext, 0) + 1
            
            ext_str = ", ".join([f"{count} {ext}" for ext, count in extensions.items()])
            
            print(f"  {category:20} {file_count:2d} files ({ext_str})")
    
    print(f"\n📊 TOTAL FILES GENERATED: {total_files}")
    
    # Key findings summary
    if sequestration_rates is not None:
        print(f"\n🔍 KEY FINDINGS:")
        print(f"  • Average carbon stock: {gdf['carbon_stock'].mean():.3f} kg/m²")
        print(f"  • Average sequestration rate: {sequestration_rates['annual_sequestration'].mean():.4f} kg/m²/yr")
        print(f"  • Total study area: ~{len(gdf) * 0.01:.2f} km²")
        print(f"  • Estimated annual sequestration: {sequestration_rates['annual_sequestration'].sum() * 10000 / 1000:.1f} tonnes/yr")
    
    print(f"\n🚀 NEXT STEPS:")
    print(f"  1. Open interactive maps in web browser")
    print(f"  2. Import GIS data into your preferred software")
    print(f"  3. Share animations and static maps in reports")
    print(f"  4. Use exported data for further analysis")
    
    print(f"\n💡 TIPS FOR USE:")
    print(f"  • Interactive HTML maps can be shared online")
    print(f"  • GeoTIFF files maintain spatial reference")
    print(f"  • Shapefiles are compatible with most GIS software")
    print(f"  • Animations are great for presentations")
    
    print("\n" + "="*80)
    print("✅ MAPPING AND VISUALIZATION COMPLETED SUCCESSFULLY!")
    print("="*80)

# Create final summary
create_visualization_gallery()